.. _graph-visualization:

# Graph Visualization

## Overview

Toyplot now includes support for visualizing graphs - in the mathematical sense of vertices connected by edges - using the :meth:`toyplot.canvas.Canvas.graph` and :func:`toyplot.graph` functions.  As we will see, graph visualizations combine many of the aspects and properties of line plots (for drawing the edges), scatterplots (for drawing the vertices), and text (for drawing labels).

At a minimum, a graph can be specified as a collection of edges.  For example, consider a trivial social network:

In [1]:
sources = ["Tim", "Tim", "Fred", "Janet"]
targets = ["Fred", "Janet", "Janet", "Pam"]

... here, we have specified a sequence of source (start) vertices and target (end) vertices for each edge in the graph, which we can pass directly to Toyplot for rendering:

In [2]:
import toyplot
toyplot.graph(sources, targets, width=300);

Fred Janet Pam Tim Save as .csv

Simple as it is, Toyplot had to perform many steps to arrive at this figure:

* We specified a set of edges as input, and Toyplot *induced* a set of unique vertices from them.
* Used a *layout algorithm* to calculate coordinates for each vertex.
* Rendered the *vertices*.
* Rendered a set of *vertex labels*.
* Rendered an *edge* (line) between each pair of connected vertices.

We will examine each of these concepts in depth over the course of this guide.

## Inputs

## Layout Algorithms

## Vertex Rendering

## Vertex Labels

## Edge Rendering